In [ ]:
#| default_exp chatbot

% pip install -q langchain
% pip install -q langchain_community
% pip install -qU langchain-openai

In [1]:
# import getpass
# import os

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [2]:
# import getpass
# import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass()

# from langchain_openai import ChatOpenAI

# model = ChatOpenAI(model="gpt-3.5-turbo")

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage

from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [4]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

helperPrompt  = """You are a job search assistant. 
You are helping a job searcher find jobs to apply for. From the list of jobs you have, you need to help the job searcher find the jobs that are relevant to them. You can ask the job searcher questions to help you understand what they are looking for.
"""

basePrompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            helperPrompt,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = basePrompt | model


In [5]:
requirementsPrompt  = """Current search requirements are: {
    "Job Title": "Software Engineer",
    "Location": "Remote",
    "Company": "Google"
}
"""

searchResultsPrompt = """
Current search results are: [
    {
        "Job Title": "Software Engineer",
        "Location": "Remote",
        "Company": "Google",
        "Status": "Pending"
    },
    {
        "Job Title": "Software Engineer",
        "Location": "Onsite",
        "Company": "Facebook",
        "Status": "Pending"
    },
    {
        "Job Title": "Software Engineer",
        "Location": "Remote",
        "Company": "Amazon",
        "Status": "Pending"
    }
]
"""



In [6]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages")


In [7]:
config = {"configurable": {"session_id": "abc4"}}

In [8]:
def remove_previous_RAG_info(session_id: str) -> dict:
    history = get_session_history(session_id)

    alteredHistory = []

    for message in history.messages:
        if type(message) is tuple:
            msgType, text = message
            if "Current search requirements are" not in text:
                if "Current search results are" not in text:
                    alteredHistory.append(message)    
        elif type(message) is HumanMessage:
            alteredHistory.append(message)
        else:
            alteredHistory.append(message)

    history.messages = alteredHistory

    return {"session_id": session_id, "messages": alteredHistory}

In [9]:
from hn_jobs_chat.requirementsBot import handleSubmit as requirementsBotResponse

def getResponse(input:str):
    remove_previous_RAG_info("abc4")

    requirementsPrompt = requirementsBotResponse(input)

    response = with_message_history.invoke(
        { "messages": [
            (   "system",
                requirementsPrompt
            ),
            (   "system",
                searchResultsPrompt
            ),
            HumanMessage(content=input)]
        },
        config=config,
    )

    return response.content

In [10]:
import ipywidgets as widgets
from IPython.display import display

# Create input widgets
text_input = widgets.Text()

# Create a button
submit_button = widgets.Button(description='Submit')

# Define a function to handle form submission
def handle_submit(button):
    response = getResponse(text_input.value)
    print(response)

# Attach the function to the button's on_click event
submit_button.on_click(handle_submit)

# Display the form
display(text_input, submit_button)



Text(value='')

Button(description='Submit', style=ButtonStyle())

input I want to work with React.
response {
  "new_requirements": {
    "tech_stack": "React"
  },
  "remove_requirements": {}
}
jsonResponse {'new_requirements': {'tech_stack': 'React'}, 'remove_requirements': {}}
Great to know that you are looking to work with React! I have found some job opportunities that might interest you. Here are the details:

1. **Job Title:** Software Engineer
   - **Location:** Remote
   - **Company:** Google

2. **Job Title:** Software Engineer
   - **Location:** Remote
   - **Company:** Amazon

Would you like more details about any of these positions or should I continue searching for more React-related opportunities?
input I don't want to work with React.
response {
  "new_requirements": {
  },
  "remove_requirements": {
    "tech_stack": "React"
  }
}
jsonResponse {'new_requirements': {}, 'remove_requirements': {'tech_stack': 'React'}}
Got it, I'll exclude React-related positions from the search. Here are some job opportunities that might interest you ba